In [1]:
import os 
os.environ["OPENAI_API_KEY"] = ""

In [2]:
from langchain import SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import create_sql_agent
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chat_models import ChatOpenAI

openAI_llm = ChatOpenAI(temperature=0.8, model_name="gpt-3.5-turbo-0613")

db = SQLDatabase.from_uri("sqlite:///.././northwind-SQLite3/dist/northwind.db")
db_chain = SQLDatabaseChain.from_llm(openAI_llm, db, verbose=True)

In [3]:
search = SerpAPIWrapper(serpapi_api_key="")

In [4]:
tools = [
    Tool(
        name="Company-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about company. Input should be in the form of a question containing full context"
    ),
    Tool(
        name = "Search-Compare",
        func=search.run,
        description="useful for when you need to answer questions about comparison. This tool should be used at last compared to other tools."
    )
]

agent = initialize_agent(tools, openAI_llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


In [5]:
agent.run("which product give the highest value for our company. Please give me more detail about the product. How is the product price compared worldwide?")




> Entering new  chain...

Invoking: `Company-DB` with `highest value product`




> Entering new  chain...
highest value product
SQLQuery:SELECT "ProductName", MAX("UnitPrice") AS "HighestPrice" FROM "Products"
SQLResult: [('Côte de Blaye', 263.5)]
Answer:Côte de Blaye
> Finished chain.
Côte de Blaye
Invoking: `Search-Compare` with `Côte de Blaye`


Strong and fruity for a young wine, with musk and spice notes developing with age. Taste. Soft and balanced tannic structure. Aging potential. 1 - 5 years ...The product that gives the highest value for our company is the "Côte de Blaye" wine. This wine is known for its strong and fruity taste, with musk and spice notes that develop with age. It has a soft and balanced tannic structure. The aging potential of this wine is 1 to 5 years.

As for the price comparison worldwide, I'm sorry but I couldn't find the specific information about the price of Côte de Blaye wine compared to other wines globally. It would be best to consult a wine mark

'The product that gives the highest value for our company is the "Côte de Blaye" wine. This wine is known for its strong and fruity taste, with musk and spice notes that develop with age. It has a soft and balanced tannic structure. The aging potential of this wine is 1 to 5 years.\n\nAs for the price comparison worldwide, I\'m sorry but I couldn\'t find the specific information about the price of Côte de Blaye wine compared to other wines globally. It would be best to consult a wine market analysis or contact wine distributors for more detailed information on worldwide price comparisons.'